# How to use

1. Run Installation and other setup
2. Open up Choose files and tools, and choose your files and tools
3. Run "Run Tools" which will do all your data processing
4. Run plotting to get graphs

<hr>

# Installation and other setup
(takes about 2 minutes)

## Installation

In [ ]:
#@title Install MiniConda (~40s)
import sys
import os

# ensure cwd is /content
os.chdir("/content")

# Install MiniConda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local

# Add the path to the conda package installation location
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [ ]:
#@title Install opensim with Conda (~1min)

# Install Opensim
!conda install -c opensim-org opensim -y
import opensim as osim

In [ ]:
#@title Importing Misc Libraries
from google.colab import files #to upload user's files
import requests #to get github files
import numpy as np #used in some functions
import time
import shutil #for moving files to same directory for tools

#for fancy graphing:
import pandas as pd
!pip install pygwalker
import pygwalker as pyg

## Common function definitions

### C3D Conversion Functions

In [ ]:
#@title class 'C3DFileAdapter'

class C3DFileAdapter(osim.FileAdapter):
    r"""
     C3DFileAdapter reads a C3D file into markers and forces tables of type
    TimeSeriesTableVec3. The markers table has each column labeled by its
    corresponding marker name. For the forces table, the data are grouped
    by sensor (force-plate #) in force, point and moment order, with the
    respective *f#*, *p#* and *m#* column labels. C3DFileAdpater provides
    options for expressing the force-plate measurements either as the
    net force and moments expressed at the ForcePlateOrigin, the
    CenterOfPressure, or the PointOfWrenchApplication.
    """

    thisown = property(lambda x: x.this.own(), lambda x, v: x.this.own(v), doc="The membership flag")

    ForceLocation_OriginOfForcePlate = osim._common.C3DFileAdapter_ForceLocation_OriginOfForcePlate
    r""" 0 : the origin of the force-plate"""
    ForceLocation_CenterOfPressure = osim._common.C3DFileAdapter_ForceLocation_CenterOfPressure
    r""" 1 : the center of pressure"""
    ForceLocation_PointOfWrenchApplication = osim._common.C3DFileAdapter_ForceLocation_PointOfWrenchApplication
    r""" 2 : PWA as defined by Shimba, 1984"""

    def clone(self) -> "OpenSim::C3DFileAdapter *":
        return osim._common.C3DFileAdapter_clone(self)

    def getLocationForForceExpression(self) -> "OpenSim::C3DFileAdapter::ForceLocation":
        r""" Retrieve the option for location for force expression"""
        return osim._common.C3DFileAdapter_getLocationForForceExpression(self)

    def getMarkersTable(self, tables: "StdMapStringAbstractDataTable") -> "std::shared_ptr< OpenSim::TimeSeriesTableVec3 >":
        r""" Retrieve the TimeSeriesTableVec3 of Markers"""
        return osim._common.C3DFileAdapter_getMarkersTable(self, tables)

    def getForcesTable(self, tables: "StdMapStringAbstractDataTable") -> "std::shared_ptr< OpenSim::TimeSeriesTableVec3 >":
        r""" Retrieve the TimeSeriesTableVec3 of Forces"""
        return osim._common.C3DFileAdapter_getForcesTable(self, tables)

    def setLocationForForceExpression(self, *args) -> "void":
        return osim._common.C3DFileAdapter_setLocationForForceExpression(self, *args)

    def __init__(self):
        osim._common.C3DFileAdapter_swiginit(self, osim._common.new_C3DFileAdapter())

# Register C3DFileAdapter in _common:
osim._common.C3DFileAdapter_swigregister(C3DFileAdapter)
C3DFileAdapter._markers = osim._common.cvar.C3DFileAdapter__markers
C3DFileAdapter._forces = osim._common.cvar.C3DFileAdapter__forces

In [ ]:
#@title 'osimC3D.m' in Python

# https://github.com/opensim-org/opensim-core/blob/master/Bindings/Java/Matlab/Utilities/osimC3D.m
class osimC3D:

  def __init__(self,path2c3d,ForceLocation):
    if not os.path.isfile(path2c3d):
      raise Exception('File does not exist. Check path is correct')
    else:
      head_tail = os.path.split(path2c3d)
#      print(head_tail)
      path = head_tail[0]
      root_ext = os.path.splitext(head_tail[1])
      name = root_ext[0]
      ext = [root_ext[1],'.trc','.mot']
 #     print(ext)
      if not path:
        raise Exception('Input path must be full path to file (example: /content/Walking.mot)')
      self.__path = path
      self.__name = name
      self.__ext = ext
    if (not (type(ForceLocation) == int)) or (not (ForceLocation >= 0 and ForceLocation <=2)):
      raise Exception('ForceLocation must be an integer of value 0, 1, or 2')

    # Use a c3dAdapter to read the c3d file
    c3dAdapter = osim.C3DFileAdapter()
    c3dAdapter.setLocationForForceExpression(ForceLocation)
    tables = c3dAdapter.read(path2c3d)#osim.c3dAdapter.read(path2c3d)
    # Set the marker and force data into OpenSim tables
    self.__markers = c3dAdapter.getMarkersTable(tables)
    self.__forces = c3dAdapter.getForcesTable(tables)
    # Set the force location specifier in case someone wants to check.
    if ForceLocation == 0:
      location = 'OriginOfForcePlate'
    elif ForceLocation == 1:
      location = 'CenterOfPressure'
    elif ForceLocation == 2:
      location = 'PointOfWrenchApplication'

    self.__ForceLocation = location;


  def getPath(self):
    return self.__path

  def getName(self):
    return self.__name

  def getForceLocation(self):
    # Get the Force Location
    return self.__ForceLocation

  def getRate_marker(self):
    # Get the capture rate used for the Marker Data
    return float(self.__markers.getTableMetaDataAsString('DataRate'))

  def getRate_force(self):
    # Get the capture rate used for the Force Data
    return float(self.__forces.getTableMetaDataAsString('DataRate'))

  def getNumTrajectories(self):
    # Get the number of markers in the c3d file
    return self.__markers.getNumColumns()

  def getNumForces(self):
    # Get the number of forceplates in the c3d file
    return (self.__forces.getNumColumns())/3

  def getStartTime(self):
    # Get the start time of the c3d file
    return self.__markers.getIndependentColumn().get(0)

  def getEndTime(self):
    # Get the end time of the c3d file
    return self.__markers.getIndependentColumn().get(self.__markers.getNumRows() - 1)

  def getFileName(self):
    # Get the name of the c3d file
    # I think the original MATLAB file may have written this function wrong, so I wrote this function according to my idea
    return self.__name+self.__ext

  def getDirectory(self):
    # Get the directory path for the c3d file
    # Again, I think the original MATLAB file may have written this function wrong, so I wrote this function according to my idea
    return self.__path+self.__name+self.__ext

  def getTable_markers(self):
    return self.__markers().clone()

  def getTable_forces(self):
    return self.__forces.clone() #self.forces().clone()

  def getAsDicts(self):
    # Convert the OpenSim tables into Pyhton Dicts
    markerDict = osimTableToDict(self.__markers);
    forcesDict = osimTableToDict(self.__forces);
    print('Maker and force data returned as Pyhton Dicts')
    return markerDict, forcesDict

  def rotateData(self,axis,value):
    # Method for rotating marker and force data stored in osim tables.
    # c3d.rotateData('x', 90)

    if not (len(axis)==0 and axis.isalpha()):
      raise Exception('Axis must be either x,y or z')

    if not type(value) == int or float:
      raise Exception('value must be numeric (90, -90, 270, ...')

    # rotate the tables
    self.rotateTable(self.__markers, axis, value);
    self.rotateTable(self.__forces, axis, value);
    print('Marker and Force tables have been rotated')

  def convertMillimeters2Meters(self):
    # Function to convert  point data (mm) to m and Torque data
    # (Nmm) to Nm.

    nRows  = self.__forces.getNumRows();
    labels = self.__forces.getColumnLabels();

    for i in range(0,self.__forces.getNumColumns):
      # All force columns will have the 'f' prefix while point
      # and moment columns will have 'p' and 'm' prefixes, respectively.
      if not labels.get(i)[0] == 'f':
        columnData = self.__forces.updDependentColumnAtIndex(i)
        for u in range(0,nRows):
          # Divide by 1000
          columnData.setRowAtIndex(u,columnData.get(u).scalarDivideEq(1000))

    print('Point and Torque values convert from mm and Nmm to m and Nm, respectively')

  def writeTRC(self,varargin):
    # Write marker data to trc file.
    # osimC3d.writeTRC()                       Write to dir of input c3d.
    # osimC3d.writeTRC('Walking.trc')          Write to dir of input c3d with defined file name.
    # osimC3d.writeTRC('C:/data/Walking.trc')  Write to defined path input path.

    # Compute an output path to use for writing to file
    outputPath = self.__generateOutputPath(varargin,'.trc')
    #print(outputPath)
    # Write to file
    osim.TRCFileAdapter().write(self.__markers, outputPath)
    print('Marker file written to '+outputPath)


  def writeMOT(self,varargin):
    # Write force data to mot file.
    # osimC3d.writeMOT()                       Write to dir of input c3d.
    # osimC3d.writeMOT('Walking.mot')          Write to dir of input c3d with defined file name.
    # osimC3d.writeMOT('C:/data/Walking.mot')  Write to defined path input path.

    # Compute an output path to use for writing to file

    outputPath = self.__generateOutputPath(varargin,'.mot')


    # Get the forces table
    forces = self.getTable_forces()
    #print(forces) #seems to look right except the headers before the column headers are weird ('can't convert to string')

    # Get the column labels
    labels = forces.getColumnLabels()
    #print("Labels info:")
    #print(type(labels)) #is tuple
    #print(labels) # ('f1', 'p1', 'm1', 'f2', 'p2', 'm2')
    #print(len(labels)) #shows 6 (is it only 6 because it's grouping the xyz's under each?)

    # Make a copy
    #updlabels = labels
    updlabels=[]

    # Labels from C3DFileAdapter are f1, p1, m1, f2,...
    # We edit them to be consistent with requirements of viewing
    # forces in the GUI (ground_force_vx, ground_force_px,...)
    for i in range(0,len(labels)):
      # Get the label as a string
      label = str(labels[i])
      # Transform the label depending on force, point, or moment
      if not label.find('f') == -1:
        label = label.replace('f', 'ground_force_')
        label = label+'_v'
      elif not label.find('p') == -1:
        label = label.replace('p', 'ground_force_')
        label = label+'_p'
      elif not label.find('m') == -1:
        label = label.replace('m', 'ground_moment_')
        label = label+'_m'

      # update the label name
      updlabels.append(label)
    updlabels=tuple(updlabels) #now we have the corrected tuple

    # set the column labels
    forces.setColumnLabels(updlabels)
    #forces.setColumnLabels(labels) #just to see where code gets up to with it in numpy form (it passes)

    # Flatten the Vec3 force table
    postfix = osim.StdVectorString()
    postfix.append('x') #changed .add to .append
    postfix.append('y')
    postfix.append('z')
    forces_flat = forces.flatten(postfix)

    # Change the header in the file to meet Storage conditions

    #changed forces_flat.getTableMetaDataKeys().size() to len(forces_flat.getTableMetaDataKeys())
    if len(forces_flat.getTableMetaDataKeys()) > 0:
      for i in range(0,len(forces_flat.getTableMetaDataKeys())):
        # Get the metakey string at index zero. Since the array gets smaller on
        # each loop, we just need to keep taking the first one in the array.

        metakey = str(forces_flat.getTableMetaDataKeys()[0]) #changef .get(0) to [0]
        # Remove the key from the meta data
        forces_flat.removeTableMetaDataKey(metakey)


    # Add the column and row data to the meta key
    forces_flat.addTableMetaDataString('nColumns',str(forces_flat.getNumColumns()+1))
    forces_flat.addTableMetaDataString('nRows',str(forces_flat.getNumRows()));

    # Write to file
    osim.STOFileAdapter().write(forces_flat, outputPath) #added .osim
    print('Forces file written to '+outputPath)

######################### Below is private functions


  def __setMarkers(self, a):
    # Private Method for setting the internal Marker table
    self.__markers = a

  def __setForces(self, a):
    # Private Method for setting the internal Force table
    self.__forces = a

  def __rotateTable(self, table, axisString, value):
    # Private Method for doing the table rotation operations

    # set up the transform
    if axisString == 'x':
      axis = CoordinateAxis(0)
    elif axisString == 'y':
      axis = CoordinateAxis(1)
    elif axisString == 'z':
      axis = CoordinateAxis(2)
    else:
      raise Exception('input axis must ne either x,y, or z')


    # instantiate a transform selfect. Rotation() is a Simbody class
    R = osim.Rotation(np.deg2rad(value),axis)
    # Rotation() works on each row.
    for iRow in range(0,table.getNumRows()):
      # get a row from the table
      rowVec = table.getRowAtIndex(iRow)
      # rotate each Vec3 element of row vector, rowVec, at once
      rowVec_rotated = R.multiply(rowVec)
      # overwrite row with rotated row
      table.setRowAtIndex(iRow,rowVec_rotated)



  def __generateOutputPath(self,path,ext):
    # Function to generate an output path from no, partial, or fully
    # defined user path.
    path = os.path.split(path)
    # Validate the output filename
    if len(path[0]) > 1:
      # Path selfect should be of size == 1, any larger and user
      # input multiple variables into function.
      raise Exception(str(size(path,2)) +' inputs, expecting zero or one')

    if len(path[0])==0:
      # No file path has been input, so use the path and name from
      # the c3d file.
      filepath = self.getPath()
      name = self.getName()
    #  print(filepath)
    #  print(name)
    else:
      if not (len(path[0])==0):# and path[0].isalpha()):
        print(len(path[0])==0)
        print(path[0].isalpha())
        raise Exception('Input must be a sting of characters')

      if path[1].find(ext) == -1:
        print(path)
        print(ext)
        print(path[1].find(ext))
        raise Exception('Input must be a path to a '+ext+' file')

      # User has included a path to write to
      head_tail = os.path.split(path[0])
      filepath = head_tail[0]
      root_ext = os.path.splitext(head_tail[1])
      name = root_ext[0]
      e = root_ext[1]
      if not filepath:
        # Only the file name is given
        filepath = self.getPath()

    #Generate the output path.
    outputPath = os.path.join(filepath, name+ext)
    return outputPath



In [ ]:
#@title Function to move CAREN c3d file to 'requirements' folder

def uploadc3d(filename):
  src = os.path.join('/home/CAREN_OpenSim_Tools-master/Example/Subject01/c3d_Files', filename)  # Get the path of the source file
  print(src)
  dst_folder = "/home/CAREN_OpenSim_Tools-master/Example/requirements"  # Replace with the path of the destination folder
  dst = os.path.join(dst_folder, filename)  # Get the path of the destination file
  print(dst)

  #os.rename(src, dst)
  shutil.copy2(src, dst) #(to COPY files to 'requirements', so code can be rerun without errors)
  files = (os.listdir(os.getcwd()))
  return files


### Static Optimization function

In [ ]:
#@title Function copied from OpenColab script
#slightly edited file path stuff because had errors
def perform_so(model_file, ik_file, grf_file, grf_xml, reserve_actuators, results_dir):
    """Performs Static Optimization using OpenSim.

    Parameters
    ----------
    model_file: str
        OpenSim model (.osim)
    ik_file: str
        kinematics calculated from Inverse Kinematics
    grf_file: str
        the ground reaction forces
    grf_xml: str
        xml description containing how to apply the GRF forces
    reserve_actuators: str
        path to the reserve actuator .xml file
    results_dir: str
        directory to store the results
    """
    # model
    model = osim.Model(model_file)

    # prepare external forces xml file
    name = os.path.basename(grf_file)[:-8]
    external_loads = osim.ExternalLoads(grf_xml, True)
    # external_loads.setExternalLoadsModelKinematicsFileName(ik_file)
    external_loads.setDataFileName(grf_file)
    # external_loads.setLowpassCutoffFrequencyForLoadKinematics(6)
    external_loads.printToXML(results_dir + name + '.xml')

    # add reserve actuators
    force_set = osim.SetForces(reserve_actuators, True)
    force_set.setMemoryOwner(False)  # model will be the owner
    for i in range(0, force_set.getSize()):
        model.updForceSet().append(force_set.get(i))

    # construct static optimization
    motion = osim.Storage(ik_file)
    static_optimization = osim.StaticOptimization()
    static_optimization.setStartTime(motion.getFirstTime())
    static_optimization.setEndTime(motion.getLastTime())
    static_optimization.setUseModelForceSet(True)
    static_optimization.setUseMusclePhysiology(True)
    static_optimization.setActivationExponent(2)
    static_optimization.setConvergenceCriterion(0.0001)
    static_optimization.setMaxIterations(100)
    model.addAnalysis(static_optimization)

    # analysis
    analysis = osim.AnalyzeTool(model)
    analysis.setName(name)
    analysis.setModel(model)
    analysis.setInitialTime(motion.getFirstTime())
    analysis.setFinalTime(motion.getLastTime())
    analysis.setLowpassCutoffFrequency(6)
    analysis.setCoordinatesFileName(ik_file)
    analysis.setExternalLoadsFileName(results_dir + name + '.xml')
    analysis.setLoadModelAndInput(True)
    analysis.setResultsDir(results_dir)
    analysis.run()
    #so_force_file = results_dir + name + '_so_forces.sto'
    #so_activations_file = results_dir + name + '_so_activations.sto'
    so_force_file = results_dir + '/' + name + '_StaticOptimization_force.sto' #EDITED, will need to generalise anyway
    so_activations_file = results_dir + '/' + name + '_StaticOptimization_activation.sto' #EDITED, need to generalise anyway
    return (so_force_file, so_activations_file)

### Reading files and graphing

In [ ]:
#@title Function to graph results files

#function to skip through file until header to ensure files are structured similarly for read_csv function
def find_header_length(filename):
    """
    This function reads the input file line by line and determines the length of the header by looking for the 'endheader' keyword.

    Parameters:
        filename (str): The name of the file to read.

    Returns:
        int: The number of lines to skip when reading the file.
    """
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            if 'endheader' in line:
                return i+1  # return number of lines to skip
    return 0  # return 0 if endheader not found


#function to determine separator of file (since it's different for different files)
def detect_separator(filename):
    with open(filename, 'r') as f:
        lines = [next(f) for x in range(30)]  # read the first this many lines of file

    separators = [',', '\t', ';']  # list of potential separators
    separator_counts = {sep: 0 for sep in separators}  # initialize counts to 0

    for line in lines:
        for sep in separators:
            separator_counts[sep] += line.count(sep)  # count frequency of separator in line

    separator = max(separator_counts, key=separator_counts.get)  # find separator with highest count

    return separator


#function to graph file (once separator is known)
def f2g(file):
  start_time = time.time()

  #find delimiter so that read_csv can read file:
  separator = detect_separator(file)

  #determine how many lines to skip to prevent errors trying to read header:
  header_len = find_header_length(file)

  #turn file into df:
  df=pd.read_csv(file, sep=separator, skiprows=header_len)
  #print(df.head())


  #downsample dataframe so you don't exceed free Colab RAM:

  #find number of rows
  num_rows = len(df)
  #print(df.shape)

  #max number of rows df can have before it's too big
  max_rows=10000   #arbitrary value that looks good enough when plotted

  #if df has too many rows, downsample until it has the max number of rows
  if num_rows > max_rows:
    # compute the ratio you want to downsample by
    ratio = int(len(df) / max_rows)
    df = df.iloc[::ratio]


  #print('df time taken: ','{:.2f} seconds'.format(time.time()-start_time))

  #now plot using pygwalker:
  #start_time = time.time()
  #print(df.shape)
  pyg.walk(df)
  #print('pyg time taken: ','{:.2f} seconds'.format(time.time()-start_time))

In [ ]:
#@title Function to find all names of dependency files of scale tool
import re

def findfiles(instance):

  #get dump string
  dump=instance.dump()

  # Define the regular expression to extract the file name
  regex = r"<(\w+_file)>(?!.*output).*?</\1>"

  # find all matches
  matches = re.findall(regex, dump)
  #print(matches)

  # Iterate over the tags and extract the values using regular expressions
  filenames=[]
  for tag in matches:
    pattern = r'<{}>(.*?)</{}>'.format(tag, tag)
    match = re.search(pattern, dump)
    if match:
        value = match.group(1).strip()
        filenames.append(value)
        #print(value)

  #remove double-ups
  filenames=list(set(filenames))

  return filenames

In [ ]:
#@title Function to copy all necessary files for scale tool into same folder

def organise_st_files(scale_setup_xml,destination_folder):

  files_needed=[]

  # create instance of each tool and find all the files needed
  scale_tool=osim.ScaleTool(scale_setup_xml)
  files_needed=findfiles(scale_tool)
  #print(files_needed)

  #hack to add in ones it missed (NEED TO FIX!!)
  files_needed+=["subject01_Scale_ScaleSet.xml","MU2392_Arms_Scale_MeasurementSet.xml"]

  # search for and relocate all files to tools_io folder

  # Define the folder you want to search in
  search_folder = '/home'

  # Define the folder you want to copy the files to
  copy_folder = destination_folder

  # Loop through each file name and search for it in the folder and subfolders
  for file_name in files_needed:
      for root, dirs, files in os.walk(search_folder):
          if file_name in files:
              # If the file exists, check if it exists in the copy folder
              src_file = os.path.join(root, file_name)
              dst_file = os.path.join(copy_folder, file_name)
              if os.path.exists(dst_file):
                  print(f"{file_name} already exists in {copy_folder} and was not relocated again")
              else:
                  shutil.copy2(src_file, dst_file)
                  print(f"Copied {file_name} to {copy_folder}")
              break
      else:
          # If the file doesn't exist, print an error message
          print(f"Error: {file_name} not found")

In [ ]:
#@title Function to copy setup CAREN files into same folder

def organise_files(setup_xml,destination_folder):

  files_needed=[]

  # create instance of each tool and find all the files needed
  tool_inst=osim.ScaleTool(setup_xml)
  files_needed=findfiles(tool_inst)
  #print(files_needed)

  #hack to add in ones it missed (will need to be generalised for other data sets)
  if setup_xml == destination_folder + 'Scale_Setup.xml':
    files_needed+=["subject01_Scale_ScaleSet.xml","MU2392_Arms_Scale_MeasurementSet.xml"]
  #elif setup_xml == destination_folder + 'IK_Setup.xml':

  # search for and relocate all files to tools_io folder

  # Define the folder you want to search in
  search_folder = '/home'

  # Define the folder you want to copy the files to
  copy_folder = destination_folder

  # Loop through each file name and search for it in the folder and subfolders
  for file_name in files_needed:
      for root, dirs, files in os.walk(search_folder):
          if file_name in files:
              # If the file exists, check if it exists in the copy folder
              src_file = os.path.join(root, file_name)
              dst_file = os.path.join(copy_folder, file_name)
              if os.path.exists(dst_file):
                  print(f"{file_name} already exists in {copy_folder} and was not relocated again")
              else:
                  shutil.copy2(src_file, dst_file)
                  print(f"Copied {file_name} to {copy_folder}")
              break
      else:
          # If the file doesn't exist, print an error message
          print(f"Error: {file_name} not found")

# Choose files and tools

In [ ]:
#@title Choose Tools
#@markdown Choose the tool you would like to run the pipeline up to (inclusive):

Tool='Inverse Kinematics' #@param ["Inverse Kinematics", "Inverse Dynamics", "Residual Reduction Algorithm", "Static Optimisation", "Computed Muscle Control"]

#@markdown Note: Any plotted quantities will be those created by the last tool executed.
#@markdown <br> <br> Also note: The CAREN setup has some problem that doesn't allow the Computed Muscle Control tool to execute properly.

if Tool =='Inverse Kinematics':
  Tool=1
elif Tool=="Inverse Dynamics":
  Tool=2
elif Tool=="Residual Reduction Algorithm":
  Tool=3
elif Tool=="Static Optimisation":
  Tool=4
elif Tool=="Computed Muscle Control":
  Tool=5

In [ ]:
#@title Choose files

save_folder='/home'

os.chdir(save_folder)

Mode='Demo' #@param ['Demo', 'CAREN_OpenSim_Tools-master.zip']

if Mode=='Demo':

  # Create directory for demo files (if it doesn't already exist)
  if not os.path.isdir(save_folder + '/Demo'):
    os.makedirs('Demo')

  #reassign 'folder' to demo directory and make cwd
  folder=save_folder+'/Demo'
  os.chdir(folder)

  # Download files:

  # Model file
  !wget -nc https://raw.githubusercontent.com/hmok/OpenColab/main/gait2354_simbody.osim
  model=os.path.join(folder,'gait2354_simbody.osim')

  # Model scaling files
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_Scale_MarkerSet.xml
  scale_markerset_xml=os.path.join(folder,'/home/gait2354_Scale_MarkerSet.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_Setup_Scale.xml
  scale_setup_xml=os.path.join(folder,'subject01_Setup_Scale.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_static.trc
  scale_trc=os.path.join(folder,'subject01_static.trc')

  # Inverse kinematics files
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_walk1.trc
  IK_trc=os.path.join(folder,'subject01_walk1.trc')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_Setup_IK.xml
  IK_setup_xml=os.path.join(folder,'subject01_Setup_IK.xml')

  # Inverse dynamics files
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_Setup_InverseDynamics.xml
  ID_setup_xml=os.path.join(folder,'subject01_Setup_InverseDynamics.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_walk1_grf.xml
  ID_grf_xml=os.path.join(folder,'subject01_walk1_grf.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_walk1_grf.mot
  ID_grf_mot=os.path.join(folder,'subject01_walk1_grf.mot')

  # RRA files
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_Actuators.xml
  RRA_actuators_xml=os.path.join(folder,'gait2354_RRA_Actuators.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_Tasks.xml
  RRA_tasks_xml=os.path.join(folder,'gait2354_RRA_Tasks.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_Setup_RRA.xml
  RRA_setup_xml=os.path.join(folder,'subject01_Setup_RRA.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_ControlConstraints.xml

  # SO files
  !wget -nc https://raw.githubusercontent.com/ESJiang/2354_result/master/Reserve_Actuators.xml
  SO_xml=os.path.join(folder,'Reserve_Actuators.xml')

  # CMC files
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_CMC_ControlConstraints.xml
  CMC_constraints_xml=os.path.join(folder,'gait2354_CMC_ControlConstraints.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_CMC_Actuators.xml
  CMC_actuators_xml=os.path.join(folder,'gait2354_CMC_Actuators.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_CMC_Tasks.xml
  CMC_tasks_xml=os.path.join(folder,'gait2354_CMC_Tasks.xml')
  !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/subject01_Setup_CMC.xml
  CMC_setup_xml=os.path.join(folder,'subject01_Setup_CMC.xml')


elif Mode=='CAREN_OpenSim_Tools-master.zip':
    # Choose file input method
    print('Choose how you want to get CAREN_OpenSim_Tools-master.zip into workspace. Enter:')
    print('d : Google Drive')
    print('a : Zip file is already uploaded')
    user_choice=None
    while user_choice not in ["d", "f", "a"]:
      user_choice = input("\nSelect your import method: ")

    if user_choice=="a":
      if os.path.exists(save_folder + '/CAREN_OpenSim_Tools-master'):
        os.chdir(save_folder + '/CAREN_OpenSim_Tools-master')
        print('Zip file is already loaded.')
      else:
        print('Could not find zip file. Import it again.')

    elif user_choice=="d":
      from google.colab import drive
      drive.mount('/content/drive')
      zip_file_path1 = '/content/drive/MyDrive/' + input("\nEnter path of Drive folder in which CAREN_OpenSim_Tools-master.zip and Requirement.zip are located: MyDrive/")
      zip_file_path2 = zip_file_path1 + '/Requirement.zip'
      zip_file_path1 += '/CAREN_OpenSim_Tools-master.zip'
      # uni/Internship/OpenColab
      #print(zip_file_path1)
    import zipfile

    with zipfile.ZipFile(zip_file_path1, 'r') as zip_ref:
      zip_ref.extractall(save_folder)

    with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
      zip_ref.extractall(save_folder)

    #set cwd to the CAREN extracted zip file
    os.chdir("/home/CAREN_OpenSim_Tools-master")
    print(os.listdir(os.getcwd()))
    #reassign 'folder' to location of tool inputs and outputs
    #(note: this is because tools won't work if input files aren't in cwd)
    #easiest to just use the 'requirements' folder in the CAREN folder
    folder = os.path.join(save_folder, 'CAREN_OpenSim_Tools-master/Example/requirements')

    print(os.listdir(os.getcwd()))
    os.chdir(folder)


    # Define setup file for each tool
    scale_setup_xml = folder + '/Scale_Setup.xml'
    IK_setup_xml = folder + '/IK_Setup.xml'
    ID_setup_xml = folder + '/ID_Setup.xml'
    RRA_setup_xml = folder + '/RRA_Setup.xml'
    SO_xml = folder + '/SO_Setup.xml'
    CMC_setup_xml = folder + '/CMC_Setup.xml'
    print(os.listdir(os.getcwd()))

    # Relocate files needed by tools to a single folder
    #organise_files(scale_setup_xml, folder) #don't use this because we want to use result of c3d conversion
    #scaletool:
    !cp /home/Requirement/ScalingXMLs_TargetSearch/gait2392_simbodyHBM.osim /home/CAREN_OpenSim_Tools-master/Example/requirements/gait2392_simbodyHBM.osim
    !cp /home/Requirement/ScalingXMLs_TargetSearch/markersHBM.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/markersHBM.xml

    #RRA tool:
    !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_Actuators.xml
    #!cp /home/Requirement/FDXMLs_TargetSearch/subject01_walk1_grf.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/subject01_walk1_grf.xml
    !cp /home/Requirement/RRAXMLs_TargetSearch/GRF.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/subject01_walk1_grf.xml #not needed?
    !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_Tasks.xml
    !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_RRA_ControlConstraints.xml

    #CMC Tool
    !wget -nc https://raw.githubusercontent.com/opensim-org/opensim-models/master/Pipelines/Gait2354_Simbody/gait2354_CMC_Actuators.xml

# Run Tools

In [ ]:
#@title Run Tools (takes a while)
os.chdir(folder)
successful='succesfully executed.'
errormessage='did not execute properly.'
finaltime=1.5 #seconds

# Convert c3d files to trc and mot
if Mode == 'CAREN_OpenSim_Tools-master.zip':
  start_time = time.time()
  print('C3D conversion:')

  # move CAREN c3d files to cwd
  uploadc3d('S+T01.c3d')
  uploadc3d('W1+T01.c3d')
  uploadc3d('Calibration.c3d')

  #make calibration.trc file
  statfile='Calibration.c3d'
  c3dfile = os.getcwd() + '/' + statfile
  conv = osimC3D(c3dfile,0)
  conv.writeTRC('Calibration.trc')

  #convert c3d file you're wanting to visualise
  c3d_basename='S+T01'
  c3dfile = os.getcwd() + '/' + c3d_basename + '.c3d'
  conv = osimC3D(c3dfile,0)
  conv.writeTRC('makes no difference, should fix')
  conv.writeMOT('makes no difference, should fix')
  #rename mot so it can be used by tool (needs to be generalised, probably by fixing the write function above)
  !cp /home/CAREN_OpenSim_Tools-master/Example/requirements/S+T01.mot /home/CAREN_OpenSim_Tools-master/Example/requirements/W101.mot
  print('Time taken: {:.2f} seconds'.format(time.time()-start_time))
  print('')


# Scale model to fit subject
start_time = time.time()
scale_tool=osim.ScaleTool(scale_setup_xml)
if scale_tool.run() == True:
  print('Scale Tool', successful, '({:.2f} seconds)'.format(time.time()-start_time) )
else:
  print('Scale Tool\n', errormessage)


# Using Inverse Kinematics on the model to produce a motion file
start_time = time.time()
ikTool = osim.InverseKinematicsTool(IK_setup_xml)
if Mode == 'CAREN_OpenSim_Tools-master.zip':
  #set up tool for CAREN
  #!cp /home/CAREN_OpenSim_Tools-master/Example/Subject01/trc_Files/S+T01.trc /home/CAREN_OpenSim_Tools-master/Example/requirements/S+T01.trc #don't need because we're making it now from c3d file
  ikTool.set_model_file('subject01_simbody.osim')
  ikTool.setMarkerDataFileName('Calibration.trc')
  ikTool.set_output_motion_file('subject01_walk1_ik.mot')
if ikTool.run() == True:
  IK_output_mot = ikTool.getPropertyByName('output_motion_file').toString()
  print('Inverse Kinematics Tool', successful, '({:.2f} seconds)'.format(time.time()-start_time))
else:
  print('/nInverse Kinematics Tool/n', errormessage)


# Inverse Dynamics
if Tool >= 2:
  start_time = time.time()
  idTool=osim.InverseDynamicsTool(ID_setup_xml)
  idTool.set_results_directory(folder)
  if Mode == 'CAREN_OpenSim_Tools-master.zip':
    #set up tool for CAREN
    idTool.setModelFileName('subject01_simbody.osim')
  if idTool.run() == True:
    ID_output_sto=idTool.getOutputGenForceFileName()
    print('Inverse Dynamics Tool', successful, '({:.2f} seconds)'.format(time.time()-start_time))
  else:
    print('\nInverse Dynamics Tool\n', errormessage)


# Residual Reduction Algorithm
if Tool >= 3:
  start_time = time.time()
  rraTool=osim.RRATool(RRA_setup_xml)
  #if Mode == 'CAREN_OpenSim_Tools-master.zip':
    #rraTool.setFinalTime(finaltime)
  if rraTool.run() == True:
    rra_path=os.path.join(os.getcwd(), rraTool.getResultsDir(), rraTool.getName())
    RRA_angles=rra_path + '_Kinematics_q.sto' #**NOTE: this assumes the tool is following a naming convention, won't work if it actually isn't
    RRA_muscleforces=rra_path + '_Actuation_force.sto' #same here
    model_adj = rraTool.getOutputModelFileName()
    print('Residual Reduction Algorithm', successful, '({:.2f} seconds)'.format(time.time() - start_time))
  else:
    print('\nResidual Reduction Algorithm\n', errormessage)


# Static Optimisation function
if Tool>=4:
  start_time = time.time()
  if Mode == 'CAREN_OpenSim_Tools-master.zip':
    model = os.path.join(folder,'subject01_simbody.osim')
    ID_grf_mot= '/home/CAREN_OpenSim_Tools-master/Example/requirements/W101.mot' #os.path.join(folder,'subject01_walk1_grf.mot') #need to find
    ID_grf_xml=os.path.join(folder,'subject01_walk1_grf.xml')

  SO_output=perform_so(model_adj, os.path.join(folder, IK_output_mot), ID_grf_mot, ID_grf_xml, SO_xml, folder)
  SO_muscleforces=SO_output[0]
  SO_generalised_ofm=SO_output[1] #generalised output muscle force
  print('Static Optimisation function', successful, '({:.2f} seconds)'.format(time.time()-start_time))


# Computed muscle control
if Tool>=5:
  start_time = time.time()
  from opensim import CMCTool
  cmcTool=osim.CMCTool(CMC_setup_xml)
  if Mode == 'CAREN_OpenSim_Tools-master.zip':
    #cmcTool.setFinalTime(finaltime)
    # need to rename external loads file to subject01_externalForces.xml for tool
    !cp /home/CAREN_OpenSim_Tools-master/Example/requirements/subject01_walk1_grf.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/subject01_externalForces.xml
    # need to copy some files to tool's wd
    !cp /home/CAREN_OpenSim_Tools-master/Example/requirements/ResultsRRA/subject01_walk1_RRA_Kinematics_q.sto /home/CAREN_OpenSim_Tools-master/Example/requirements/subject01_walk1_RRA_Kinematics_q.sto
    !cp /home/Requirement/CMCXMLs_TargetSearch/gait2354_CMC_Tasks.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/gait2354_CMC_Tasks.xml
    !cp /home/Requirement/CMCXMLs_TargetSearch/gait2354_CMC_ControlConstraints.xml /home/CAREN_OpenSim_Tools-master/Example/requirements/gait2354_CMC_ControlConstraints.xml
  if cmcTool.run() == True:
    cmc_path=os.path.join(os.getcwd(), cmcTool.getResultsDir()[2:], cmcTool.getName())
    CMC_angles=cmc_path + '_Kinematics_q.sto' #**NOTE: this assumes the tool is following a naming convention, won't work if it actually isn't
    CMC_muscleforces=cmc_path + '_Actuation_force.sto' #same here
    print('Computed Muscle Control Tool', successful, '({:.2f} seconds)'.format(time.time()-start_time))
  else:
    print('\nComputed Muscle Control Tool\n', errormessage)

# Plotting

In [ ]:
#@title Joint Angles

if Tool>=5:
  print('Displaying angles generated by the Computed Muscle Control tool.\n')
  f2g(CMC_angles)
elif Tool>=3:
  print('Displaying angles generated by the Residual Reduction Algorithm tool.\n')
  f2g(RRA_angles)
else:
  print('Displaying angles generated by the Inverse Kinematics tool.\n')
  f2g(IK_output_mot)

In [ ]:
#@title Joint Moments

if Tool>=5:
  print('Displaying moments generated by the Computed Muscle Control tool.\n')
  f2g(CMC_muscleforces)
elif Tool>=3:
  print('Displaying moments generated by the Residual Reduction Algorithm tool.\n')
  f2g(RRA_muscleforces)
elif Tool>=2:
  print('Displaying angles generated by the Inverse Kinematics tool.\n')
  f2g(ID_output_sto)
else:
  print('No graphs generated because pipeline has not been run up to at least the inverse dynamics tool.')

In [ ]:
#@title Muscle Forces (generated by static optimisation function)

if Tool>=4:
  print('Muscle Forces:')
  f2g(SO_muscleforces)

  print('Generalised Muscle Forces:')
  f2g(SO_generalised_ofm)

else:
  print('No graphs generated because static optimisation function not run.')

In [ ]:
#@title Custom (specify the filename you want to graph)

#filename='subject01_walk1_ik.mot'
#f2g(filename)